In [16]:
from keras.regularizers import l1,l2
from keras.models import Sequential,Model
from keras.layers import Embedding,Input,Dense,Flatten,Lambda, Multiply, Concatenate


In [ ]:
def get_model(num_users,num_items,mf_dim=10,layers=[10],reg_layers=[0,0,0,0],reg_mf=0):
    num_layer=len(layers)

    user_input=Input(shape=(1,),dtype='int32')
    item_input_pos=Input(shape=(1,),dtype='int32')
    item_input_neg = Input(shape=(1,), dtype='int32')

    ## in this context each point is projected to have 10 dimensional coordinates?
    MF_embedding_user=Embedding(input_dim=num_users,output_dim=mf_dim,embeddings_initializer='random_normal',
                                name='mf_user_embedding',embeddings_regularizer=l2(reg_mf),input_length=1)
    MF_embedding_item = Embedding(input_dim=num_items, output_dim=mf_dim, embeddings_initializer='random_normal',
                                  name='mf_item_embedding',embeddings_regularizer=l2(reg_mf), input_length=1)
    MLP_embedding_user=Embedding(input_dim=num_users,output_dim=layers[0],embeddings_initializer='random_normal',
                                 name='mlp_user_embedding', embeddings_regularizer=l2(reg_mf),input_length=1)
    MLP_embedding_item = Embedding(input_dim=num_items, output_dim=layers[0], embeddings_initializer='random_normal',
                                   name='mlp_item_embedding',embeddings_regularizer=l2(reg_mf), input_length=1)

    mf_user_latent=Flatten()(MF_embedding_user(user_input))
    mf_item_latent_pos=Flatten()(MF_embedding_item(item_input_pos))
    mf_item_latent_neg = Flatten()(MF_embedding_item(item_input_neg))

    ## merge = deprecated use keras.layers.Concatenate(axis=-1) instead
    prefer_pos = merge([mf_user_latent, mf_item_latent_pos], mode='mul')
    prefer_neg = merge([mf_user_latent, mf_item_latent_neg], mode='mul')
    ## convert negative layer to negative, lambda layers should be re-written as subclass layer if too complex (eg: multiply by scale?)
    prefer_neg = Lambda(lambda x: -x)(prefer_neg)
    ## basically just merge 2 layer
    mf_vector = merge([prefer_pos, prefer_neg], mode='concat')

    ## flat matrix to an array
    mlp_user_latent=Flatten()(MLP_embedding_user(user_input))
    mlp_item_latent_pos=Flatten()(MLP_embedding_item(item_input_pos))
    mlp_item_latent_neg=Flatten()(MLP_embedding_item(item_input_neg))
    mlp_item_latent_neg=Lambda(lambda x:-x)(mlp_item_latent_neg)
    mlp_vector=merge([mlp_user_latent,mlp_item_latent_pos,mlp_item_latent_neg],mode='concat')
    for idx in range(1,num_layer):
        #set up hidden layer of the network, why tanh and have to regularize?? L1 consider weight, l2 consider square of weight
        layer=Dense(layers[idx],kernel_regularizer=l2(0.0000),activation='tanh',name="layer%d" %idx)
        mlp_vector=layer(mlp_vector)

    ## concatenation of NBPR layer and DNCR layer
    predict_vector=merge([mf_vector,mlp_vector],mode='concat')


    #set up prediction --> final layer, sigmoid activate to give binary output
    prediction=Dense(1,activation='sigmoid',kernel_initializer='lecun_uniform',name='prediction')(predict_vector)
    model=Model(inputs=[user_input,item_input_pos,item_input_neg],outputs=prediction)

    return model

In [5]:
user_input=Input(shape=(1,),dtype='int32')
item_input_pos=Input(shape=(1,),dtype='int32')
item_input_neg = Input(shape=(1,), dtype='int32')

In [7]:
mf_dim=10
layers=[10]
num_layer=len(layers)
reg_layers=[0,0,0,0]
reg_mf=0

In [14]:
MF_embedding_user=Embedding(input_dim=2000,output_dim=mf_dim,embeddings_initializer='random_normal',
                                name='mf_user_embedding',embeddings_regularizer=l2(reg_mf),input_length=1)
MF_embedding_item = Embedding(input_dim=50000, output_dim=mf_dim, embeddings_initializer='random_normal',
                                  name='mf_item_embedding',embeddings_regularizer=l2(reg_mf), input_length=1)
MLP_embedding_user=Embedding(input_dim=2000,output_dim=layers[0],embeddings_initializer='random_normal',
                                 name='mlp_user_embedding', embeddings_regularizer=l2(reg_mf),input_length=1)
MLP_embedding_item = Embedding(input_dim=50000, output_dim=layers[0], embeddings_initializer='random_normal',
                                   name='mlp_item_embedding',embeddings_regularizer=l2(reg_mf), input_length=1)

In [15]:
mf_user_latent=Flatten()(MF_embedding_user(user_input))
mf_item_latent_pos=Flatten()(MF_embedding_item(item_input_pos))
mf_item_latent_neg = Flatten()(MF_embedding_item(item_input_neg))

2022-12-14 23:27:03.991416: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
## merge = deprecated use keras.layers.Concatenate(axis=-1) instead
prefer_pos = Multiply()([mf_user_latent, mf_item_latent_pos])
prefer_neg = Multiply()([mf_user_latent, mf_item_latent_neg])
## convert negative layer to negative, lambda layers should be re-written as subclass layer if too complex (eg: multiply by scale?)
prefer_neg = Lambda(lambda x: -x)(prefer_neg)
## basically just merge 2 layer
mf_vector = Concatenate()([prefer_pos, prefer_neg])

In [27]:
mlp_user_latent=Flatten()(MLP_embedding_user(user_input))
mlp_item_latent_pos=Flatten()(MLP_embedding_item(item_input_pos))
mlp_item_latent_neg=Flatten()(MLP_embedding_item(item_input_neg))
mlp_item_latent_neg=Lambda(lambda x:-x)(mlp_item_latent_neg)
mlp_vector=Concatenate()([mlp_user_latent,mlp_item_latent_pos,mlp_item_latent_neg])
for idx in range(1,num_layer):
    #set up hidden layer of the network, why tanh and have to regularize?? L1 consider weight, l2 consider square of weight
    layer=Dense(layers[idx],kernel_regularizer=l2(0.0000),activation='tanh',name="layer%d" %idx)
    mlp_vector=layer(mlp_vector)

In [29]:
predict_vector=Concatenate()([mf_vector,mlp_vector])


#set up prediction --> final layer, sigmoid activate to give binary output
prediction=Dense(1,kernel_initializer='lecun_uniform',name='prediction')(predict_vector)
model=Model(inputs=[user_input,item_input_pos,item_input_neg],outputs=prediction)

In [33]:
predict_vector

<KerasTensor: shape=(None, 50) dtype=float32 (created by layer 'concatenate_3')>

In [35]:
mf_user_latent

<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'flatten')>

In [12]:
mat = sp.dok_matrix((2000, 50000), dtype=np.float32)
mat

<2000x50000 sparse matrix of type '<class 'numpy.float32'>'
	with 0 stored elements in Dictionary Of Keys format>

In [36]:
layers

[10]

In [25]:
num_layer=len(layers)

In [26]:
num_layer

1